Prepare our HTTP API client

In [46]:
module HttpClient =
    open System 
    open System.Net.Http 
    open System.Net.Http.Headers
    open System.IO

    type RequestMethod = 
        | Get
        | Post 

    let client = new System.Net.Http.HttpClient()

    let buildQueryFromMap (parameters: Map<string, string>) =
        use content = new FormUrlEncodedContent(parameters)
        let stream = content.ReadAsStream()
        let reader = new StreamReader(stream)
        reader.ReadToEnd()

    let handleResponseAsync req = 
        async {
            let! response = client.SendAsync(req) |> Async.AwaitTask
            // response.EnsureSuccessStatusCode() |> ignore 

            return!
                response.Content.ReadAsStringAsync()
                |> Async.AwaitTask
        }

    let getRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)

        handleResponseAsync req


    let postRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) (someBody: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)

        match someBody with 
        | Some body -> 
            req.Content <- new FormUrlEncodedContent(body)
        | None -> 
            ()

        handleResponseAsync req

Define our Travel module which find coordiate from place

In [47]:
module Travel = 
    // Not sure why it has this problem while in Elixir Livebook, it doesn't have this problem.
    let locateFromNominatim place = 
        let queryOptions:  Map<string, string> = 
            Map.empty
                .Add("q", $"{place}")
                .Add("format", "json")
                .Add("addressdetails", "1")
        let headerOptions: Map<string, string> = 
            Map.empty
                .Add("accept", "application/json")

        async {
            return! HttpClient.getRequestAsync "https://nominatim.openstreetmap.org/search" (Some headerOptions) (Some queryOptions)
        }
        
    let locateFromBing place = 
        let bingMapApiKey = "AoCXJTYP_sdzAo9lji9UnjEfqvmdLJoC0qUCoxEPCKcVBGgCi0wyFmvSJP7Y9o6T"
        let apiUrl = "https://dev.virtualearth.net/REST/v1/Locations"

        let queryOptions:  Map<string, string> = 
            Map.empty
                .Add("q", $"{place}")
                .Add("key", bingMapApiKey)
        let headerOptions: Map<string, string> = 
            Map.empty
                .Add("accept", "application/json")

        async {
            return! HttpClient.getRequestAsync apiUrl (Some headerOptions) (Some queryOptions)
        }

        
        

In [48]:
Travel.locateFromNominatim "丹东市, 振兴区" |> Async.RunSynchronously

<html>
<head>
<title>Access blocked</title>
</head>
<body>
<h1>Access blocked</h1>

<p>You have been blocked because you have violated the
<a href="https://operations.osmfoundation.org/policies/nominatim/">usage policy</a>
of OSM's Nominatim geocoding service. Please be aware that OSM's resources are
limited and shared between many users. The usage policy is there to ensure that
the service remains usable for everybody.</p>

<p>Please review the terms and make sure that your
software adheres to the terms. You should in particular verify that you have set a
<b>custom HTTP referrer or HTTP user agent</b> that identifies your application, and
that you are not overusing the service with massive bulk requests.</p>

<p>If you feel that this block is unjustified or remains after you have adopted
your usage, you may contact the Nominatim system administrator at
nominatim@openstreetmap.org to have this block lifted.</p>
</body>
</head>


Decode something like this:
 """
        {
            "authenticationResultCode": "ValidCredentials",
            "brandLogoUri": "http://dev.virtualearth.net/Branding/logo_powered_by.png",
            "copyright": "Copyright ? 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.",
            "resourceSets": [
                {
                    "estimatedTotal": 1,
                    "resources": [
                        {
                            "__type": "Location:http://schemas.microsoft.com/search/local/ws/rest/v1",
                            "bbox": [
                                39.817195892333984,
                                124.12994384765625,
                                40.156288146972656,
                                124.40470123291016
                            ],
                            "name": "Zhenxing District, China",
                            "point": {
                                "type": "Point",
                                "coordinates": [
                                    40.12990189,
                                    124.3833313
                                ]
                            },
                            "address": {
                                "adminDistrict": "Liaoning",
                                "adminDistrict2": "Dandong",
                                "countryRegion": "China",
                                "formattedAddress": "Zhenxing District, China",
                                "locality": "Zhenxing District"
                            },
                            "confidence": "High",
                            "entityType": "PopulatedPlace",
                            "geocodePoints": [
                                {
                                    "type": "Point",
                                    "coordinates": [
                                        40.12990189,
                                        124.3833313
                                    ],
                                    "calculationMethod": "Rooftop",
                                    "usageTypes": [
                                        "Display"
                                    ]
                                }
                            ],
                            "matchCodes": [
                                "Good"
                            ]
                        }
                    ]
                }
            ],
            "statusCode": 200,
            "statusDescription": "OK",
            "traceId": "65af3aeeb18d429abe886249bed877bd|CO00004BAE|0.0.0.1|Ref A: 047C4CDF114448729D5E2078D4C5735B Ref B: CO1EDGE1208 Ref C: 2023-03-06T13:43:55Z"
        }
        """

In [49]:
#load @"../../.paket/load/net6.0/Thoth.Json.Net.fsx"
open Thoth.Json.Net

type Cooridate = {
    lat: float 
    lon: float
}


type TravelPlace = {
    name: string
    coordinate: Cooridate
}


let coordinateDecoder: Decoder<Cooridate> = 
    Decode.object (
        fun get -> 
            {
                let cooridates = 
                    (get.Required.Field "coordinates" (Decode.list float))
                    |> Array.ofList

                Cooridate.lat = coordiates[0]
                Cooridate.lon = coordiates[1]

            }
    )

let (|GetCoordiateFromStr|_|) pointStr: Option<Cooridate option> = 
    match pointStr |> Decode.fromString (Decode)



let mapDecoder: Decoder<TravelPlace> = 
    Decode.object (
        fun get -> 
        {
            TravelPlace.name = get.Required.Field "name" Decode.string
            TravelPlace.lat = get.
        }
    )

let responseStr = Travel.locateFromBing "丹东市, 振兴区" |> Async.RunSynchronously
printfn "%A" responseStr


// // Extract the coordinates from the response object
// let coordinates =
//     match responseObj.ResourceSets |> Seq.tryHead with
//     | Some(resourceSet) ->
//         match resourceSet.Resources |> Seq.tryHead with
//         | Some(resource) ->
//             match resource.Point with
//             | Some(point) -> (point.Coordinates.[0], point.Coordinates.[1])
//             | None -> failwith "No coordinates found in response"
//         | None -> failwith "No resources found in response"
//     | None -> failwith "No resource sets found in response"

// // Process the coordinates here
// printfn "Latitude: %f, Longitude: %f" (fst coordinates) (snd coordinates)

"{"authenticationResultCode":"ValidCredentials","brandLogoUri":"http:\/\/dev.virtualearth.net\/Branding\/logo_powered_by.png","copyright":"Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.","resourceSets":[{"estimatedTotal":1,"resources":[{"__type":"Location:http:\/\/schemas.microsoft.com\/search\/local\/ws\/rest\/v1","bbox":[39.817195892333984,124.12994384765625,40.156288146972656,124.40470123291016],"name":"Zhenxing District, China","point":{"type":"Point","coordinates":[40.12990189,124.3833313]},"address":{"adminDistrict":"Liaoning","adminDistrict2":"Dandong","countryRegion":"China","formattedAddress":"Zhenxing District, China","locality":"Zhenxing District"},"confidence":"High","entityType":"PopulatedPlace","geocodePoints":[{"type":"Point","coordinates":[40.12990189,124.3833313],"calculat